In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from itertools import product
from sklearn.model_selection import train_test_split, TimeSeriesSplit

# Load dataset
data = pd.read_csv('master2.csv')
data['date'] = pd.to_datetime(data['date'])
data = data.sort_values(by='date')

# Encode date-related features as sine and cosine transformations
data['month'] = data['date'].dt.month
data['day_of_year'] = data['date'].dt.dayofyear
data['month_sin'] = np.sin(2 * np.pi * data['month'] / 12)
data['month_cos'] = np.cos(2 * np.pi * data['month'] / 12)
data['day_sin'] = np.sin(2 * np.pi * data['day_of_year'] / 365)
data['day_cos'] = np.cos(2 * np.pi * data['day_of_year'] / 365)

# Add lag features
data['lag1'] = data['water_area_km2'].shift(1)
data['lag3'] = data['water_area_km2'].shift(3)
data['lag7'] = data['water_area_km2'].shift(7)
data['lag14'] = data['water_area_km2'].shift(14)

# Feature engineering
data['rolling_avg_7'] = data['water_area_km2'].rolling(window=7).mean()
data['rolling_median_7'] = data['water_area_km2'].rolling(window=7).median()
data['rolling_std_7'] = data['water_area_km2'].rolling(window=7).std()
data['expanding_mean'] = data['water_area_km2'].expanding().mean()
data['cumulative_rainfall'] = data['Rainfall'].cumsum()
data['rainfall_ndwi_interaction'] = data['Rainfall'] * data['mean_ndwi']
data['extreme_rainfall'] = (data['Rainfall'] > data['Rainfall'].quantile(0.95)).astype(int)

# Calculate monthly aggregates
data['month_year'] = data['date'].dt.to_period('M')
monthly_aggregates = data.groupby('month_year')[['water_area_km2']].agg(['mean', 'max']).reset_index()
monthly_aggregates.columns = ['month_year', 'monthly_mean', 'monthly_max']
data = pd.merge(data, monthly_aggregates, how='left', on='month_year')

# Handle missing values
data = data.fillna(method='ffill')

# Train-test split (80/20 split)
train_data = data.iloc[:int(len(data) * 0.7)].dropna()
test_data = data.iloc[int(len(data) * 0.7):].dropna()

# Prepare data for Prophet
prophet_train = train_data.rename(columns={'date': 'ds', 'water_area_km2': 'y'})
prophet_test = test_data.rename(columns={'date': 'ds', 'water_area_km2': 'y'})

# Define additional regressors
additional_regressors = [
    'Average_Temperature', 'Rainfall', 'lag1', 'lag3', 'lag7', 'lag14',
    'rolling_avg_7', 'rolling_median_7', 'rolling_std_7', 'expanding_mean',
    'cumulative_rainfall', 'rainfall_ndwi_interaction', 'extreme_rainfall',
    'month_sin', 'month_cos', 'day_sin', 'day_cos',
    'monthly_mean', 'monthly_max','Average_Humidity','Max_Humidity','Average_Wind_Speed','Max_Wind_Speed','Max_Temperature'
]

for regressor in additional_regressors:
    if regressor in train_data.columns:
        prophet_train[regressor] = train_data[regressor]
        prophet_test[regressor] = test_data[regressor]

# Hyperparameter tuning (cross-validation)
param_grid = {
    'changepoint_prior_scale': [0.01, 0.1, 0.5, 1.0],
    'seasonality_prior_scale': [0.1, 1.0, 5.0, 10.0]
}

best_params = None
best_score = float('inf')

# Cross-validation to check for overfitting
tscv = TimeSeriesSplit(n_splits=2)

for changepoint_prior_scale, seasonality_prior_scale in product(param_grid['changepoint_prior_scale'], param_grid['seasonality_prior_scale']):
    cv_scores = []
    
    # Perform cross-validation with independent Prophet models in each fold
    for train_idx, val_idx in tscv.split(prophet_train):
        train_cv, val_cv = prophet_train.iloc[train_idx], prophet_train.iloc[val_idx]
        
        model = Prophet(
            changepoint_prior_scale=changepoint_prior_scale,
            seasonality_prior_scale=seasonality_prior_scale,
            daily_seasonality=True
        )

        for regressor in additional_regressors:
            if regressor in prophet_train.columns:
                model.add_regressor(regressor)
        
        model.fit(train_cv)
        forecast = model.predict(val_cv)
        mae = mean_absolute_error(val_cv['y'], forecast['yhat'])
        cv_scores.append(mae)

    avg_cv_score = np.mean(cv_scores)

    if avg_cv_score < best_score:
        best_score = avg_cv_score
        best_params = (changepoint_prior_scale, seasonality_prior_scale)

# Train Prophet model with optimal hyperparameters
optimal_changepoint_prior_scale, optimal_seasonality_prior_scale = best_params

prophet_model = Prophet(
    changepoint_prior_scale=optimal_changepoint_prior_scale,
    seasonality_prior_scale=optimal_seasonality_prior_scale,
    daily_seasonality=True
)

for regressor in additional_regressors:
    if regressor in prophet_train.columns:
        prophet_model.add_regressor(regressor)

prophet_model.fit(prophet_train)

# Make predictions
forecast_train = prophet_model.predict(prophet_train)
forecast_test = prophet_model.predict(prophet_test)

# Evaluate the model
def evaluate_model(y_true, y_pred, data_split):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"{data_split} - MAE: {mae:.2f}, MSE: {mse:.2f}, R²: {r2:.2f}")

evaluate_model(prophet_train['y'], forecast_train['yhat'], 'Train')
evaluate_model(prophet_test['y'], forecast_test['yhat'], 'Test')

print(f"Optimal Hyperparameters: changepoint_prior_scale={optimal_changepoint_prior_scale}, seasonality_prior_scale={optimal_seasonality_prior_scale}")

# Save Prophet predictions
forecast_test[['ds', 'yhat']].to_csv('prophet_predictions_optimized.csv', index=False)
print("Prophet predictions saved to 'prophet_predictions_optimized.csv'.")

# Prophet Visualization: Actual vs. Predicted
plt.figure(figsize=(10, 6))
plt.plot(prophet_test['ds'], prophet_test['y'], label='Actual', color='blue')
plt.plot(forecast_test['ds'], forecast_test['yhat'], label='Predicted', color='red')
plt.xlabel('Date')
plt.ylabel('Water Area (km²)')
plt.title('Prophet: Actual vs Predicted')
plt.legend()
plt.grid()
plt.show()

# Prophet trend components
prophet_model.plot_components(forecast_test)
plt.show()
